In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np

In [3]:
import requests
import random
import xlrd
import csv
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

In [4]:
import matplotlib as mpl
import matplotlib.style
import seaborn as sns  
import matplotlib.pyplot as plt

In [6]:
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA

In [9]:
# setting to default parameters
plt.rcParams.update(plt.rcParamsDefault)

# formatting for decimal places
pd.set_option("display.float_format", "{:.2f}".format)


In [12]:

# matplotlib settings
mpl.rcParams.update(mpl.rcParamsDefault)
#plt.style.use('whitegrid')
mpl.rcParams["figure.figsize"] = (12, 8)
mpl.rcParams["axes.grid"] = False

In [13]:
# setting seed for model reproducibility
seed_value = 42
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

In [15]:
# setting the destination for the data folder
path = os.path.join(os.getcwd(), "C:/Users/ASUS/Petroleum-Production-Engineering/src/data")
norm_path = os.path.normpath(path)

In [16]:
# defining a function to scrape NDIC data
# https://www.dmr.nd.gov/oilgas/
# data from May 2015 to December 2018 will be used as a training dataset
# data from 2019 will be used as a test dataset